In [ ]:
# Import libraries
import os
import numpy as np
import MDAnalysis as mda
import nmrformd as nmrmd

recalculate=True

# For figure plotting
import sys
from matplotlib import pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
msize = 15
fontsize = 30
font = {'family': 'sans', 'color':  'black', 'weight': 'normal', 'size': fontsize}
myblue_light = [42/ 255, 99/255, 255/ 255] # [0/ 255, 150/255, 177/ 255]
myblue_dark = [43/ 255, 140/255, 238/ 255] # [0/ 255, 150/255, 177/ 255]
myred= [244/ 255, 143/255, 97/ 255] # [255/ 255, 77/255, 77/ 255]
mygreen = [122/255, 190/255, 165/255]
lightgray = [0.1, 0.1, 0.1]
darkgray = [0.9, 0.9, 0.9]
white = [0, 0, 0]
black = [1, 1, 1]
sys.path.append('../../shared/')
from tools import figure_improvement, save_figure

# Path to data
numbers_mol = np.int32(np.logspace(np.log10(25), np.log10(4000), 12))
datapath = "../../raw-data/bulk-water/"

In [ ]:
scalar_properties = []
intra_gij = []
intra_R1 = []
intra_R2 = []
intra_J = []
inter_gij = []
inter_R1 = []
inter_R2 = []
inter_J = []

for number_mol in numbers_mol:
    if recalculate:
        u = None
        data = datapath+"N"+str(number_mol)+"/"
        if os.path.exists(data+"topology.data"):
            u = mda.Universe(data+"topology.data", data+"prod.xtc")
        elif os.path.exists(data+"run.tpr"):
            u = mda.Universe(data+"run.tpr", data+"run.xtc")

        if u is not None:
            timestep = np.int32(u.trajectory.dt)
            total_time = np.int32(u.trajectory.totaltime)
            n_molecules = u.atoms.n_residues
            assert n_molecules == number_mol
            v_box = np.round(np.prod(u.dimensions[:3]) / 1000,1)
            print("The number of water molecules is", n_molecules, "and the density is", np.round(n_molecules/v_box,2), "molecule per nm3")
            print("----------------------------------------------------------------------------")
            # cut-off for frequency and time
            # cut-off for frequency and time
            f_max = 1.1e5
            t_max = 2e3
            if u.trajectory.n_frames == 100001:
                results_intra = []
                results_inter = []
                for tstart, tstop in zip(np.arange(0, 100001, 10000)[:-1], np.arange(0, 100001, 10000)[1:]):
                    if os.path.exists(data+"topology.data"):
                        v = mda.Universe(data+"topology.data", data+"prod.xtc")
                        group_i = v.select_atoms("type 2")
                    elif os.path.exists(data+"run.tpr"):
                        v = mda.Universe(data+"run.tpr", data+"run.xtc")
                        group_i = v.select_atoms("type HW")
                    v.transfer_to_memory(start=tstart, stop=tstop)
                    nmr_intra = nmrmd.NMR(v, group_i, isotropic=True, type_analysis="intra_molecular", number_i=1)
                    results_intra.append(nmr_intra)
                    nmr_inter = nmrmd.NMR(v, group_i, isotropic=True, type_analysis="inter_molecular", number_i=100)
                    results_inter.append(nmr_inter)
            else:
                if os.path.exists(data+"topology.data"):
                    v = mda.Universe(data+"topology.data", data+"prod.xtc")
                    group_i = v.select_atoms("type 2")
                elif os.path.exists(data+"run.tpr"):
                    v = mda.Universe(data+"run.tpr", data+"run.xtc")
                    group_i = v.select_atoms("type HW")
                nmr_intra = nmrmd.NMR(v, group_i, isotropic=True, type_analysis="intra_molecular", number_i=300)
                results_intra.append(nmr_intra)
                nmr_inter = nmrmd.NMR(v, group_i, isotropic=True, type_analysis="inter_molecular", number_i=50)
                results_inter.append(nmr_inter)
            n = 0
            gij_intra = np.zeros(2000)
            R1_intra = np.zeros(1100)
            R2_intra = np.zeros(1100)
            J_intra = np.zeros(1100)
            gij_inter = np.zeros(2000)
            R1_inter = np.zeros(1100)
            R2_inter = np.zeros(1100)
            J_inter = np.zeros(1100)
            for result_intra, result_inter in zip(results_intra, results_inter):
                # intra
                t = result_intra.t[result_intra.t<t_max][:len(gij_intra)]
                f = result_intra.f[result_intra.f<f_max][:len(R1_intra)]
                R1_intra += result_intra.R1[result_intra.f<f_max][:len(R1_intra)]
                R2_intra += result_intra.R2[result_intra.f<f_max][:len(R2_intra)]
                J_intra += result_intra.J[0][result_intra.f<f_max][:len(J_intra)]
                gij_intra += result_intra.gij[0][result_intra.t<t_max][:len(gij_intra)]
                # inter
                t = result_inter.t[result_inter.t<t_max][:len(gij_inter)]
                f = result_inter.f[result_inter.f<f_max][:len(R1_inter)]
                R1_inter += result_inter.R1[result_inter.f<f_max][:len(R1_inter)]
                R2_inter += result_inter.R2[result_inter.f<f_max][:len(R2_inter)]
                J_inter += result_inter.J[0][result_inter.f<f_max][:len(J_inter)]
                gij_inter += result_inter.gij[0][result_inter.t<t_max][:len(gij_inter)]
                n += 1
            gij_intra /= n
            R1_intra /= n
            R2_intra /= n
            J_intra /= n
            gij_inter /= n
            R1_inter /= n
            R2_inter /= n
            J_inter /= n
            intra_gij.append(gij_intra)
            intra_R1.append(R1_intra)
            intra_R2.append(R2_intra)
            intra_J.append(J_intra)
            np.savetxt("../../treated-data/best-practice/bulk-water/gij_intra_finite_size.dat", np.vstack([t, intra_gij]).T)
            np.savetxt("../../treated-data/best-practice/bulk-water/R1_intra_finite_size.dat", np.vstack([f, intra_R1]).T)
            np.savetxt("../../treated-data/best-practice/bulk-water/R2_intra_finite_size.dat", np.vstack([f, intra_R2]).T)
            np.savetxt("../../treated-data/best-practice/bulk-water/J_intra_finite_size.dat", np.vstack([f, intra_J]).T)
            inter_gij.append(gij_inter)
            inter_R1.append(R1_inter)
            inter_R2.append(R2_inter)
            inter_J.append(J_inter)
            np.savetxt("../../treated-data/best-practice/bulk-water/gij_inter_finite_size.dat", np.vstack([t, inter_gij]).T)
            np.savetxt("../../treated-data/best-practice/bulk-water/R1_inter_finite_size.dat", np.vstack([f, inter_R1]).T)
            np.savetxt("../../treated-data/best-practice/bulk-water/R2_inter_finite_size.dat", np.vstack([f, inter_R2]).T)
            np.savetxt("../../treated-data/best-practice/bulk-water/J_inter_finite_size.dat", np.vstack([f, inter_J]).T)
            # Save volume
            scalar_properties.append([number_mol, v_box])
            np.savetxt("../../treated-data/best-practice/bulk-water/scalar_finite_size.dat", scalar_properties)
    else:
        data = np.loadtxt("../../treated-data/best-practice/bulk-water/gij_intra_finite_size.dat").T
        t, intra_gij = data[0], data[1:]
        data = np.loadtxt("../../treated-data/best-practice/bulk-water/gij_inter_finite_size.dat").T
        t, inter_gij = data[0], data[1:]
        data = np.loadtxt("../../treated-data/best-practice/bulk-water/R1_inter_finite_size.dat").T
        f, inter_R1 = data[0], data[1:]
        data = np.loadtxt("../../treated-data/best-practice/bulk-water/R1_intra_finite_size.dat").T
        f, intra_R1 = data[0], data[1:]
        data = np.loadtxt("../../treated-data/best-practice/bulk-water/R2_inter_finite_size.dat").T
        f, inter_R2 = data[0], data[1:]
        data = np.loadtxt("../../treated-data/best-practice/bulk-water/R2_intra_finite_size.dat").T
        f, intra_R2 = data[0], data[1:]
        data = np.loadtxt("../../treated-data/best-practice/bulk-water/J_inter_finite_size.dat").T
        f, inter_J = data[0], data[1:]
        data = np.loadtxt("../../treated-data/best-practice/bulk-water/J_intra_finite_size.dat").T
        f, intra_J = data[0], data[1:]
        scalar_properties = np.loadtxt("../../treated-data/best-practice/bulk-water/scalar_finite_size.dat")

In [ ]:
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):

    R = np.linspace(mygray[0], myblue[0], len(numbers_mol)//6)
    G = np.linspace(mygray[1], myblue[1], len(numbers_mol)//6)
    B = np.linspace(mygray[2], myblue[2], len(numbers_mol)//6)
    grad = np.vstack([R, G, B]).T

    fig = plt.figure(figsize=(16, 12))
    ax1 = fig.add_subplot(211)
    cpt = 0
    for inter, n_mol in zip(inter_gij[::11], scalar_properties[::11]):
        ax1.loglog(t, inter/inter[0], 'o', color=grad[cpt], label=r"V = "+str(n_mol[1])+" nm$^3$")
        cpt += 1
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=None, ylabel=r'$G^{(0)}_\textrm{inter} (t) / G^{(0)}_\textrm{inter} (0)$',
                       xlim=(1, 1000), ylim=(0.00001, 1))
    ax1 = fig.add_subplot(212)
    cpt = 0
    for intra, n_mol in zip(intra_gij[::11], scalar_properties[::11]):
        ax1.loglog(t, intra/intra[0], 'o', color=grad[cpt], label=r"V = "+str(n_mol[1])+" nm$^3$")
        cpt += 1
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$t$ (ps)", ylabel=r'$G^{(0)}_\textrm{intra} (t) / G^{(0)}_\textrm{intra} (0)$',
                       xlim=(1, 1000), ylim=(0.00001, 1))
    save_figure(plt, fig, mode, name='best-practices/size-effect-gij')

In [ ]:
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):

    R = np.linspace(mygray[0], myblue[0], len(numbers_mol)//6)
    G = np.linspace(mygray[1], myblue[1], len(numbers_mol)//6)
    B = np.linspace(mygray[2], myblue[2], len(numbers_mol)//6)
    grad = np.vstack([R, G, B]).T

    fig = plt.figure(figsize=(16, 12))
    ax1 = fig.add_subplot(211)
    cpt = 0
    for inter, n_mol in zip(inter_R1[::11], scalar_properties[::11]):
        ax1.loglog(f, inter, 'o', color=grad[cpt], label=r"V = "+str(n_mol[1])+" nm$^3$")
        cpt += 1
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=None, ylabel=r'$R_\textrm{1, inter}$ (s)',
                       xlim=(100, 100000), ylim=(0.01, 1))
    ax1 = fig.add_subplot(212)
    cpt = 0
    for intra, n_mol in zip(intra_R1[::11], scalar_properties[::11]):
        ax1.loglog(f, intra, 'o', color=grad[cpt], label=r"V = "+str(n_mol[1])+" nm$^3$")
        cpt += 1
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r"$f$ (MHz)", ylabel=r'$R_\textrm{1, intra}$ (s)',
                       xlim=(100, 100000), ylim=(0.001, 10))
    save_figure(plt, fig, mode, name='best-practices/size-effect-R1')

In [ ]:
sys.path.append("../../../nmrformd/")
from utilities import calculate_tau

In [ ]:
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):

    R = np.linspace(mygray[0], myblue[0], len(numbers_mol))
    G = np.linspace(mygray[1], myblue[1], len(numbers_mol))
    B = np.linspace(mygray[2], myblue[2], len(numbers_mol))
    grad1 = np.vstack([R, G, B]).T
    R = np.linspace(mygray[0], myred[0], len(numbers_mol))
    G = np.linspace(mygray[1], myred[1], len(numbers_mol))
    B = np.linspace(mygray[2], myred[2], len(numbers_mol))
    grad2 = np.vstack([R, G, B]).T

    fig = plt.figure(figsize=(16, 7.5))
    ax1 = fig.add_subplot(121)
    cpt = 0
    for J, gij, n_mol in zip(inter_J, inter_gij, scalar_properties):
        tau_inter = calculate_tau(J, gij, 0, oneDarray=True)
        ax1.semilogx(n_mol[1], tau_inter, 'o', color=grad1[cpt])
        cpt += 1
    ax1.semilogx(n_mol[1], tau_inter, 'o', color=grad1[cpt-1]) #, label=r"inter")
    #cpt = 0
    #for J, gij, n_mol in zip(intra_J, intra_gij, scalar_properties):
    #    tau_intra = calculate_tau(J, gij, 0, oneDarray=True)
    #    ax1.loglog(n_mol[1], tau_intra, 's', color=grad2[cpt])
    #    cpt += 1
    #ax1.loglog(n_mol[1], tau_intra, 's', color=grad2[cpt-1], label=r"intra")
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r'$V$ (nm$^3$)', ylabel=r'$\tau_\textrm{inter}$ (ps)',
                       xlim=(0.5, 200), ylim=(2, 5))

    ax1 = fig.add_subplot(122)
    cpt = 0
    for iR1, n_mol in zip(inter_R1, scalar_properties):
        tau_inter = np.mean(iR1[:10])
        ax1.semilogx(n_mol[1], tau_inter, 'o', color=grad1[cpt])
        cpt += 1
    ax1.semilogx(n_mol[1], tau_inter, 'o', color=grad1[cpt-1]) #, label=r"inter")
    #cpt = 0
    #for iR1, n_mol in zip(intra_R1, scalar_properties):
    #    tau_intra = np.mean(iR1[:10])
    #    ax1.loglog(n_mol[1], tau_intra, 's', color=grad2[cpt])
    #    cpt += 1
    #ax1.loglog(n_mol[1], tau_intra, 's', color=grad2[cpt-1], label=r"intra")
    figure_improvement(ax1, mygray, font, fontsize,
                       xlabel=r'$V$ (nm$^3$)', ylabel=r'$R_\textrm{1, inter}$ (s)',
                       xlim=(0.5, 200), ylim=(0.0, 0.3))
    save_figure(plt, fig, mode, name='best-practices/size-effect-tau-R1')